<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Изучение-исходных-данных" data-toc-modified-id="Изучение-исходных-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение исходных данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Предобработка текста</a></span></li><li><span><a href="#Подготовка-данных-для-обучения" data-toc-modified-id="Подготовка-данных-для-обучения-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Подготовка данных для обучения</a></span></li><li><span><a href="#Векторизация-текста" data-toc-modified-id="Векторизация-текста-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Векторизация текста</a></span></li></ul></li><li><span><a href="#Промежуточный-вывод" data-toc-modified-id="Промежуточный-вывод-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Промежуточный вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

# Описание данных

Столбец text в нём содержит текст комментария, а toxic — целевой признак.

# Импортирование и установка библиотек

In [21]:
!pip install catboost --q

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import lightgbm as lgb
import time
import re 
import nltk
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.utils import resample
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords

TEST_SIZE = 0.25
RANDOM_STATE = 12345

import warnings
warnings.filterwarnings("ignore")

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Методы для работы 

**Метод для очистки и нормализации текста**

In [23]:
def clean_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

**Метод для лемматизации текста**

In [24]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    return " ".join(lemmatized_tokens)

## Загрузка и подготовка данных

### Изучение исходных данных

In [25]:
data = pd.read_csv('/datasets/toxic_comments.csv')

print(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### Подготовка данных

**Удалим ненужный индексирующий столбец 'Unnamed: 0'**

In [26]:
data = data.drop(columns=['Unnamed: 0'])

print(data.columns)

Index(['text', 'toxic'], dtype='object')


#### Предобработка текста

**Очистка и нормализация текстов**

In [27]:
data['text'] = data['text'].map(clean_text)

**Лемматизация текста**

In [28]:
data['text'] = data['text'].map(lemmatize_text)

In [29]:
display(data.head())

,text,toxic
0,Explanation Why the edits make under my userna...,0
1,D aww He match this background colour I m seem...,0
2,Hey man I m really not try to edit war It s ju...,0
3,More I can t make any real suggestion on impro...,0
4,You sir be my hero Any chance you remember wha...,0


**Удаление дубликатов**

In [30]:
print('Количество дубликатов до удаления: ', data.duplicated().sum())
data.drop_duplicates(inplace = True)
print('Количество дубликатов после удаления: ', data.duplicated().sum())

Количество дубликатов до удаления:  1160
Количество дубликатов после удаления:  0


#### Подготовка данных для обучения

**Выделим целевой и остальные признаки**

In [31]:
features = data['text']
target = data['toxic']

**Разделение данных на выборки: обучающую и тестовую в соотношении 1 к 4.**

In [32]:
features_train_1, features_test_1, target_train, target_test = train_test_split(
    features, 
    target, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE, 
    stratify = target)

#### Векторизация текста 

In [33]:
try:
    nltk.download('stopwords', quiet = True)
except:
    pass
try:
    stopwords = set(stopwords.words('english'))
except:
    pass

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

features_train = count_tf_idf.fit_transform(features_train_1)
features_test = count_tf_idf.transform(features_test_1)

In [34]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(118599, 135467)
(39533, 135467)
(118599,)
(39533,)


### Промежуточный вывод

В результате этапа загрузки и подготовки данных были выполненые следующие шаги:
1. Данные были успешно загружены и изучены:
    * В датасете пропуски не наблюдаются
    * Типы данных корректны
    * Есть лишний индексирующий столбец 
2. Данные были подготовлены:
    * Удалён ненужный столбец
    * Выполнена предобработка текста в столбце 'text'
    * Данные были подготовлены для обучения моделей

**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Обучение

Для обучения возьмём такие модели, как:
* LogisticRegression
* RandomForestClassifier
* CatBoostClassifier

### LogisticRegression

In [35]:
%%time

regression = LogisticRegression(fit_intercept=True, 
                                class_weight='balanced', 
                                random_state=42,
                                solver='liblinear')

regression_parameters = {'C': [0.1, 1, 10]}

regression_grid = GridSearchCV(estimator=regression, 
                               param_grid=regression_parameters, 
                               scoring='f1', 
                               cv=3)
regression_grid.fit(features_train, target_train)

best_params = regression_grid.best_params_
best_score = regression_grid.best_score_

print(f"Лучшие параметры: {best_params}")
print(f"Среднее качество модели (F1-score) на кросс-валидации: {best_score:.4f}")
print('_______________')

regression_cv_score = cross_val_score(regression, features_train, target_train, scoring='f1', cv=3).mean()
print(f'Среднее качество модели Логистической регрессии (F1-score) с использованием cross_val_score: {regression_cv_score:.4f}')

Лучшие параметры: {'C': 10}
Среднее качество модели (F1-score) на кросс-валидации: 0.7579
_______________
Среднее качество модели Логистической регрессии (F1-score) с использованием cross_val_score: 0.7423
CPU times: user 1min 24s, sys: 1min 30s, total: 2min 55s
Wall time: 2min 55s


**Проверим качество модели на тестовых данных.**

In [36]:
%%time
from sklearn.metrics import f1_score

regression_model = LogisticRegression(fit_intercept=True,
                                      class_weight='balanced',
                                      random_state=42,
                                      solver='liblinear',
                                      C=10)

regression_model.fit(features_train, target_train)

regression_predictions = regression_model.predict(features_test)

regression_f1 = f1_score(target_test, regression_predictions)

print(f"F1-score модели Логистической регрессии на тестовых данных: {regression_f1:.3f}")

F1-score модели Логистической регрессии на тестовых данных: 0.754
CPU times: user 13.2 s, sys: 12.5 s, total: 25.7 s
Wall time: 25.7 s


### RandomForestClassifier

In [37]:
%%time

random_forest = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs = -1)

random_forest_parametrs = { 
     'n_estimators': range(20, 40, 5),
     'max_depth': range(4, 8, 2)
}

random_forest_grid = GridSearchCV(estimator=random_forest, 
                                  param_grid=random_forest_parametrs, 
                                  scoring='f1', 
                                  cv=3, 
                                  verbose=1)
random_forest_grid.fit(features_train, target_train)

best_params_rf = random_forest_grid.best_params_
best_score_rf = random_forest_grid.best_score_

print(f"Лучшие параметры: {best_params_rf}")
print(f"Среднее качество модели (F1-score) на кросс-валидации: {best_score_rf:.4f}")
print('_______________')

random_forest_cv_score = cross_val_score(random_forest, features_train, target_train, scoring='f1', cv=3).mean()
print(f'Среднее качество модели RandomForestClassifier (F1-score) с использованием cross_val_score: {random_forest_cv_score:.4f}')


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Лучшие параметры: {'max_depth': 6, 'n_estimators': 35}
Среднее качество модели (F1-score) на кросс-валидации: 0.3087
_______________
Среднее качество модели RandomForestClassifier (F1-score) с использованием cross_val_score: 0.5967
CPU times: user 28min 49s, sys: 2.25 s, total: 28min 51s
Wall time: 28min 52s


### CatBoostClassifier

**Отдельно выделим признаки и целевой признак для обучения CatBoost с меньшим количеством**

In [38]:
features_train_cb = features_train_1[:900]
features_test_cb = features_test_1[:300]
target_train_cb = target_train[:900]
target_test_cb = target_test[:300]

In [39]:
try:
    nltk.download('stopwords')
except:
    pass
try:
    stopwords = set(stopwords.words('english'))
except:
    pass
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

features_train_cb = count_tf_idf.fit_transform(features_train_cb)
features_test_cb = count_tf_idf.transform(features_test_cb)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
%%time
catboost = CatBoostClassifier(random_state=42, auto_class_weights='Balanced', verbose=False)

catboost_parameters = {
    'iterations': [100, 200],
    'learning_rate': [0.03, 0.1],
    'depth': [4, 6, 10]
}

catboost_grid = GridSearchCV(estimator=catboost, 
                             param_grid=catboost_parameters, 
                             scoring='f1', 
                             cv=3)
catboost_grid.fit(features_train_cb, target_train_cb)

best_params_cb = catboost_grid.best_params_
best_score_cb = catboost_grid.best_score_

print(f"Лучшие параметры: {best_params_cb}")
print(f"Среднее качество модели (F1-score) на кросс-валидации: {best_score_cb:.4f}")
print('_______________')

catboost_cv_score = cross_val_score(catboost, features_train_cb, target_train_cb, scoring='f1', cv=3).mean()
print(f'Среднее качество модели CatBoostClassifier (F1-score) с использованием cross_val_score: {catboost_cv_score:.4f}')

Лучшие параметры: {'depth': 6, 'iterations': 200, 'learning_rate': 0.1}
Среднее качество модели (F1-score) на кросс-валидации: 0.4593
_______________
Среднее качество модели CatBoostClassifier (F1-score) с использованием cross_val_score: 0.4555
CPU times: user 7min 54s, sys: 1.72 s, total: 7min 55s
Wall time: 8min 24s


## Выводы

**Оценивание результатов моделей**

In [42]:
columns = ['Модель', 'f1-мера на кросс-валидационной выборке']
regression_model_results = ['Логистическая регрессия', regression_cv_score]
forest_model_results = ['Случайный лес', random_forest_cv_score]
cb_model_results = ['CatBoostClassifier', catboost_cv_score]

table = pd.DataFrame([regression_model_results, forest_model_results, cb_model_results], columns = columns)

display(table)

,Модель,f1-мера на кросс-валидационной выборке
0,Логистическая регрессия,0.742311
1,Случайный лес,0.596708
2,CatBoostClassifier,0.455522


В результате выполнения проекта, были выполнены следующие этапы:

**1. Датасет был успешно загружен.**

*Общие сведения по датасету:*
* Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.
* Пропуски отсутствуют 
* Типы данных корректны

**2. Данные были подготовлены.**

Сначала была выполнена *предобработка текста:*
* Очистка и нормализация текстов
* Лемматизация текста

Далее были удалены дубликаты и выполнена *подготовка данных для обучения*:
* Данные были разделены на обучающую и тестовую выборки 
* Выполнена векторизация текста 

**3. Модели обучены.**

Для обучения были задействованы 3 модели:
* LogisticRegression
* RandomForestClassifier
* CatBoostClassifier

Как показали результаты, наилучшей моделью по метрике f1 (0.742311) является модель логистической регрессии. Следующей после неё идёт модель случайного леса, её метрика f1 меньше (0.596708). Модель CatBoostClassifier обладает наименьшей f1-мерой (0.455522).

**F1-score модели Логистической регрессии на тестовых данных: 0.754**